## day49_pyspark_withcolumn_rename

In [0]:
data = [
    (1, "Alice", "Engineering", 85000, "2020-01-15"),
    (2, "Bob", "HR", 65000, "2019-03-22"),
    (3, "Charlie", "Finance", 70000, "2018-07-10"),
    (4, "David", "Engineering", 90000, "2021-05-30"),
    (5, "Eve", "Marketing", 72000, "2017-11-12"),
    (6, "Frank", "Sales", 68000, "2016-09-18"),
    (7, "Grace", "HR", 66000, "2022-02-25"),
    (8, "Heidi", "Finance", 71000, "2015-12-01"),
    (9, "Ivan", "Engineering", 88000, "2019-08-14"),
    (10, "Judy", "Marketing", 73000, "2020-04-05")
]
columns = ["employee_id", "name", "department", "salary", "hire_date"]
df_employees = spark.createDataFrame(data, schema="employee_id int, name string, department string, salary int, hire_date string")
display(df_employees)

In [0]:
df_employees.createOrReplaceTempView("employee")

#### Add the new colmns ('country')

In [0]:
spark.sql("select *, 'India' as country from employee").display()

In PySpark, we use lit() (from pyspark.sql.functions) to add a constant or literal value as a new column in a DataFrame.

🔹 Meaning of Code

🔹 withColumn("country", ...) → creates a new column named country

🔹 lit("India") → inserts the constant value "India" in every row ( take only one value lit("India"))

In [0]:
from pyspark.sql.functions import lit
df_employees.withColumn("country", lit("India")).display()

In [0]:
df_employees.display()

## pyspark df(Data frameis ) mutable or immutable?



---

### ❓Q1: What does *immutable* mean in PySpark DataFrames?

**✅ Answer:**
In PySpark, **DataFrames are immutable**, meaning once created, their data **cannot be changed or updated directly**.
Any transformation (like `withColumn`, `drop`, `select`, etc.) creates a **new DataFrame**, without modifying the original one.

---

### ❓Q2: What happens when we add a new column using `withColumn()`?

**✅ Answer:**
When you use:

```python
df_new = df_employees.withColumn("country", lit("India"))
```

* It creates a **new DataFrame (`df_new`)** with the extra column `country`.
* The **original DataFrame (`df_employees`)** remains unchanged.
* This is because PySpark does not update data in place.

---

### ❓Q3: Why do we say transformations in PySpark are temporary?

**✅ Answer:**
Because transformations are **lazy** and **only applied in memory** during execution.
They don’t modify the actual data source or file — they just show a new view of data in Spark memory.
To make changes permanent, you must **write** the new DataFrame back (e.g., `.write.save()` or `.write.parquet()`).

---

### ❓Q4: How is this different from Pandas?

**✅ Answer:**
In **Pandas**, you can directly modify data using assignment:

```python
df['country'] = 'India'
```

This updates the original `df`.
But in **PySpark**, the same operation must create a **new DataFrame** — the old one stays unchanged.

---

### ❓Q5: How do we keep the updated data permanently?

**✅ Answer:**
You must **assign it back** or **save it**:

```python
df_employees = df_employees.withColumn("country", lit("India"))
```

or

```python
df_employees.write.parquet("path/to/save/")
```

This way, the updated version replaces or saves the modified dataset.

---

### 🧠 Summary in One Line:

> “In PySpark, every transformation creates a new DataFrame — because DataFrames are immutable.
> If you want to keep the changes, you must assign or save the new DataFrame explicitly.”

---

Would you like me to also give this explanation in **Gujarati + English mix** for easier interview understanding?


In [0]:
# 10 % bounus add 
df_employees.createOrReplaceTempView("employee")
spark.sql("select *, salary*0.1 as bonus, salary+(salary*0.1) as total_salary from employee ").display()

In [0]:
from pyspark.sql.functions import col
# spark.sql("select *, salary*0.1 as bonus, salary+(salary*0.1) as total_salary from employee ").display()

df_employees.withColumn("bonus", col("salary")*0.1).withColumn("total_salary", col("salary")+(col("salary")*0.1)).display()



In [0]:
df_employees.display()

In [0]:
df_employees.createOrReplaceTempView("employee")

In [0]:
spark.sql("select * from employee").display()

In [0]:
spark.sql("""select employee_id, 
          upper(name) as name, 
          department, 
          salary, 
          hire_date       
          from employee""").display()

In [0]:
# spark.sql("""select employee_id, 
#           upper(col(name)) as name, 
#           department, 
#           salary, 
#           hire_date       
#           from employee""").display()

from pyspark.sql.functions import *

df_employees = df_employees.withColumnRenamed("name", "First Name")
display(df_employees)

#### Case statment


In [0]:
spark.sql("""
    SELECT *,
           CASE 
               WHEN salary < 70000 THEN 'low'
               WHEN salary BETWEEN 70001 AND 75000 THEN 'mid'
               ELSE 'high'
           END AS salary_cat
    FROM employee
""").display()


In [0]:
# spark.sql("""
#     SELECT *,
#            CASE 
#                WHEN salary < 70000 THEN 'low'
#                WHEN salary BETWEEN 70001 AND 75000 THEN 'mid'
#                ELSE 'high'
#            END AS salary_cat
#     FROM employee
# """).display()


from pyspark.sql.functions import expr

df_employees = df_employees.withColumn(
    'salary_cat',
    expr("""
        case 
            when salary < 70000 then 'low'
            when salary between 70001 and 75000 then 'mid'
            else 'high'
        end
    """)
)

df_employees.show()


#--------------------------

(df_employees
    .withColumn(
        'salary_cat',
        when(col('salary') < 70000, 'low')
        .when((col('salary') >= 70001) & (col('salary') <= 75000), 'mid')
        .otherwise('high')
    )
    .display()
)



In [0]:
df_employees.display()

In [0]:
df_employees.printSchema()

In [0]:
df_employees = df_employees.withColumn("salary", col("salary").cast("float"))
df_employees.printSchema()

In [0]:
df_employees.createOrReplaceTempView("employee")

In [0]:
df_employees.display()
df_employees.show()


In [0]:
# Create or replace SQL view
df_employees.createOrReplaceTempView("employee")

# Now query using Spark SQL
spark.sql("""
    SELECT 
        employee_id,
        `First Name` AS full_name,
        department,
        salary,
        hire_date,
        salary_cat
    FROM employee
""").display()



In [0]:
from pyspark.sql.functions import col

display(
    df_employees
        .withColumnRenamed("First Name", "full_name")
        .select("employee_id", "full_name", "department")
        .filter(col("full_name") == "David")
        .withColumn("employee_id", col("employee_id") * 100)
        .withColumnRenamed("department", "dept")
)


### Drop and drop duplicates

In [0]:
df_employees.display()

In [0]:
df_employees = df_employees.drop('salary_cat','department').show()

### drop Null values   

In [0]:
data = [
    (1, "Alice", None),
    (2, None, 5000),
    (3, "Charlie", 7000),
    (4, "David", None),
    (None, "Eve", 9000)
]
columns = ["employee_id", "name", "salary"]
df_nulls = spark.createDataFrame(data, columns)
display(df_nulls)

In [0]:
df_nulls.dropna().display()

In [0]:
df_nulls.dropna(subset=['employee_id']).display()

In [0]:
df.dropDuplicates(subset=['employee_id','salary']).display()

### Drop duplicate

In [0]:

data = [
    (1, "Alice", 5000),
    (2, "Bob", 6000),
    (1, "Alice", 5000),  # duplicate row
    (3, "Charlie", 7000),
    (2, "Bob", 9000)     # duplicate row
]
columns = ["employee_id", "name", "salary"]
df = spark.createDataFrame(data, columns)
display(df)

In [0]:
df.dropDuplicates().display()

In [0]:
df.dropDuplicates(subset=['employee_id']).display()

In [0]:
df.dropDuplicates(subset=['employee_id','salary']).display()

#### Group by

In [0]:
df.groupBy('employee_id','name','salary').count().filter('count>1').display()